In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
import torch.nn as nn
import torch.optim as optim
import os

# 设备选择
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据预处理
transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.GaussianBlur(5),
])

# 数据集路径
data_dir = r"L:\常惠林\萎凋\自然萎凋\分类"

# 加载数据集（ImageFolder 自动从文件夹读取类别）
dataset = datasets.ImageFolder(root=data_dir, transform=transforms)

# 训练集 & 测试集划分
train_size = int(0.8 * len(dataset))  # 80% 训练集
test_size = len(dataset) - train_size  # 20% 测试集
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 获取类别名称
class_names = dataset.classes
num_classes = len(class_names)
print("类别:", class_names)

# 加载 VGG16 模型（预训练）
model = models.vgg16(pretrained=True)

# 修改全连接层（替换最后的分类层）
model.classifier[6] = nn.Linear(4096, num_classes) 
# 迁移到 GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
num_epochs = 100  # 训练轮数

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 保存模型
torch.save(model.state_dict(), "vgg16_tea_classification.pth")
print("训练完成，模型已保存！")

# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"测试准确率: {100 * correct / total:.2f}%")
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt




In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 设备选择
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备:", device)

# 数据预处理
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.GaussianBlur(5),
    transforms.RandomHorizontalFlip(),
])

# 数据集路径（请根据实际情况修改）
data_dir = r"L:\常惠林\萎凋\自然萎凋\分类"

# 使用 ImageFolder 自动读取数据集（文件夹名称为类别标签）
dataset = datasets.ImageFolder(root=data_dir, transform=data_transform)
print("类别:", dataset.classes)
num_classes = len(dataset.classes)

# 按 6:2:2 划分数据集
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
print(f"训练集: {train_size}, 验证集: {val_size}, 测试集: {test_size}")

# 数据加载器，batch size 为 32
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# 加载预训练的 VGG16 模型
model = models.vgg16(pretrained=True)
# 修改最后的全连接层以适应当前任务
in_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_features, num_classes)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 训练配置
num_epochs = 100
best_val_acc = 0.0

# 用于记录训练和验证指标的列表
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    running_loss = 0.0
    running_correct = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        running_correct += (preds == labels).sum().item()
        total_train += labels.size(0)
    epoch_loss = running_loss / total_train
    epoch_acc = running_correct / total_train
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)
    
    # 验证阶段
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_running_correct += (preds == labels).sum().item()
            total_val += labels.size(0)
    val_loss = val_running_loss / total_val
    val_acc = val_running_correct / total_val
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    # 保存验证集上表现最好的模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16.pth")
        print("保存最佳模型！")

# 绘制训练/验证曲线
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_loss_history, label="Train Loss")
plt.plot(val_loss_history, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_acc_history, label="Train Accuracy")
plt.plot(val_acc_history, label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()
plt.show()

# 测试阶段
model.load_state_dict(torch.load("best_vgg16.pth", map_location=device))
model.eval()
test_correct = 0
total_test = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        total_test += labels.size(0)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
test_acc = 100 * test_correct / total_test
print(f"测试准确率: {test_acc:.2f}%")

# 计算混淆矩阵并显示
cm = confusion_matrix(all_labels, all_preds)
print("混淆矩阵:")
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset.classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()
